In [1]:
% matplotlib inline

import os, sys, time, pickle, tempfile
import math, random, itertools
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from scipy.misc import logsumexp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.linalg import kron
from fastkml import kml, styles
from shapely.geometry import Point, LineString
import pulp

In [2]:
RANK_C = 10  # regularisation parameter for rankSVM
BIN_CLUSTER = 5  # number of bins/clusters for discritizing POI features
ranksvm_dir = '$HOME/work/ranksvm'
data_dir = '../data'
fpoi = os.path.join(data_dir, 'poi-Melb-all.csv')
fvisits = os.path.join(data_dir, 'userVisits-Melb.csv')
fphotos = os.path.join(data_dir, 'Melb_photos_bigbox.csv')

In [4]:
# 1. Load Data

#Load POI Data

poi_all = pd.read_csv(fpoi)
poi_all.set_index('poiID', inplace=True)
#poi_all.head()

In [5]:
poi_df = poi_all.copy()
poi_df.drop('poiURL', axis=1, inplace=True)
poi_df.rename(columns={'poiName':'Name', 'poiTheme':'Category', 'poiLat':'Latitude', 'poiLon':'Longitude'}, \
              inplace=True)
poi_df.head()

,Name,Category,Latitude,Longitude
poiID,,,,
0,Arts Precinct,City precincts,-37.82167,144.96778
1,Docklands,City precincts,-37.81700,144.94600
2,Government Precinct,City precincts,-37.81190,144.97300
3,Little Italy,City precincts,-37.79972,144.96694
4,RMIT City,City precincts,-37.80778,144.96333


In [6]:
#Load Trajectory Data

visits = pd.read_csv(fvisits, sep=';')
#visits.head()

In [7]:
visits.drop(['poiTheme', 'poiFreq'], axis=1, inplace=True)
visits.rename(columns={'seqID':'trajID'}, inplace=True)
visits.head()

,photoID,userID,dateTaken,poiID,trajID
0,3233170275,34823318@N06,949323600,30,1370
1,2104214833,49503207397@N01,1006801914,40,2340
2,2104993710,49503207397@N01,1006803152,71,2340
3,2104215119,49503207397@N01,1006803177,71,2340
4,2104994064,49503207397@N01,1006803611,71,2340


In [8]:
# 2. Compute POI Statistics DataFrame

#Compute POI visit statistics: 

#the number of photos associated with each POI:
poi_photo = visits[['photoID', 'poiID']].copy().groupby('poiID').agg(np.size)
poi_photo.rename(columns={'photoID':'#photos'}, inplace=True)
poi_photo.head()

,#photos
poiID,
0,102
1,139
2,304
3,138
4,349


In [9]:
#the visit duration at each POI:
poi_duration = visits[['dateTaken', 'poiID', 'trajID']].copy().groupby(['trajID', 'poiID']).agg([np.min, np.max])
poi_duration.columns = poi_duration.columns.droplevel()
poi_duration.rename(columns={'amin':'arrivalTime', 'amax':'departureTime'}, inplace=True)
poi_duration.reset_index(inplace=True)
poi_duration['poiDuration'] = poi_duration['departureTime'] - poi_duration['arrivalTime']
poi_duration.head()

,trajID,poiID,arrivalTime,departureTime,poiDuration
0,0,25,1226726126,1226726126,0
1,1,58,1205332532,1205332541,9
2,1,66,1205342722,1205342729,7
3,2,59,1205374109,1205374109,0
4,3,58,1205417265,1205417265,0


In [10]:
#filtering out zero visit duration at POI
poi_duration = poi_duration[poi_duration['poiDuration'] > 0]

In [11]:
#the median and summation of POI visit duration=
poi_duration_stats = poi_duration[['poiID', 'poiDuration']].copy().groupby('poiID').agg([np.median, np.sum])
poi_duration_stats.columns = poi_duration_stats.columns.droplevel()
poi_duration_stats.rename(columns={'median':'medianDuration(sec)', 'sum':'totalDuration(sec)'}, inplace=True)
poi_duration_stats.head()

,medianDuration(sec),totalDuration(sec)
poiID,,
0,332.5,24600
1,438.5,42491
2,1835.0,141155
3,1971.5,61945
4,829.0,137382


In [12]:
#the number of visits at each POI by all users and by distinct users:  

poi_visits = visits[['userID', 'trajID', 'poiID', 'photoID']].copy().groupby(['userID','trajID','poiID']).agg(np.size)
poi_visits.reset_index(inplace=True)
poi_visits.rename(columns={'photoID':'#photosAtPOIInTraj'}, inplace=True)
poi_visits.head()

,userID,trajID,poiID,#photosAtPOIInTraj
0,10058801@N06,0,25,1
1,10087938@N02,1,58,2
2,10087938@N02,1,66,2
3,10087938@N02,2,59,1
4,10087938@N02,3,58,1


In [13]:
poi_visits_stats = poi_visits[['userID', 'poiID']].copy().groupby('poiID').agg([pd.Series.nunique, np.size])
poi_visits_stats.columns = poi_visits_stats.columns.droplevel()
poi_visits_stats.rename(columns={'nunique':'#distinctUsers', 'size':'#visits'}, inplace=True)
poi_visits_stats.head()

,#distinctUsers,#visits
poiID,,
0,64,73
1,39,52
2,83,116
3,44,54
4,50,84


In [14]:
#copy visit statistics to POI dataframe:
poi_df['#photos'] = 0
poi_df['#visits'] = 0
poi_df['#distinctUsers'] = 0
poi_df['medianDuration(sec)'] = 0.0
poi_df['totalDuration(sec)'] = 0.0

In [15]:
poi_df.loc[poi_photo.index, '#photos'] = poi_photo['#photos']
poi_df.loc[poi_visits_stats.index, '#visits'] = poi_visits_stats['#visits']
poi_df.loc[poi_visits_stats.index, '#distinctUsers'] = poi_visits_stats['#distinctUsers']
poi_df.loc[poi_duration_stats.index, 'medianDuration(sec)'] = poi_duration_stats['medianDuration(sec)']
poi_df.loc[poi_duration_stats.index, 'totalDuration(sec)'] = poi_duration_stats['totalDuration(sec)']
poi_df.head()

,Name,Category,Latitude,Longitude,#photos,#visits,#distinctUsers,medianDuration(sec),totalDuration(sec)
poiID,,,,,,,,,
0,Arts Precinct,City precincts,-37.82167,144.96778,102,73,64,332.5,24600.0
1,Docklands,City precincts,-37.81700,144.94600,139,52,39,438.5,42491.0
2,Government Precinct,City precincts,-37.81190,144.97300,304,116,83,1835.0,141155.0
3,Little Italy,City precincts,-37.79972,144.96694,138,54,44,1971.5,61945.0
4,RMIT City,City precincts,-37.80778,144.96333,349,84,50,829.0,137382.0


In [16]:
#visualise & save POIs to CSV file
#poi_file = os.path.join(data_dir, 'poi_df.csv')
#poi_df.to_csv(poi_file, index=True)

In [17]:
# 3. POI vs Photo

#POIs with NO Visits


poi_df[poi_df['#visits'] < 1]

,Name,Category,Latitude,Longitude,#photos,#visits,#distinctUsers,medianDuration(sec),totalDuration(sec)
poiID,,,,,,,,,
54,Flemington Racecourse,Sports stadiums,-37.79028,144.91250,0,0,0,0.0,0.0
64,Royal Melbourne Golf Club,Sports stadiums,-37.97000,145.03000,0,0,0,0.0,0.0
87,Yarra River,Transport,-37.85194,144.90833,0,0,0,0.0,0.0


In [18]:
# 4. Compute Trajectories

#compute trajectories using POI visiting records, assuming NO loops/subtours in trajectories

traj_all = visits[['userID', 'trajID', 'poiID', 'dateTaken']].copy().groupby(['userID', 'trajID', 'poiID'])\
           .agg([np.min, np.max, np.size])
traj_all.columns = traj_all.columns.droplevel()
traj_all.reset_index(inplace=True)
traj_all.rename(columns={'amin':'startTime', 'amax':'endTime', 'size':'#photo'}, inplace=True)

In [19]:
traj_len = traj_all[['userID', 'trajID', 'poiID']].copy().groupby(['userID', 'trajID']).agg(np.size)
traj_len.reset_index(inplace=True)
traj_len.rename(columns={'poiID':'trajLen'}, inplace=True)

In [20]:
traj_all = pd.merge(traj_all, traj_len, on=['userID', 'trajID'])
traj_all['poiDuration'] = traj_all['endTime'] - traj_all['startTime']
traj_all.head(10)

,userID,trajID,poiID,startTime,endTime,#photo,trajLen,poiDuration
0,10058801@N06,0,25,1226726126,1226726126,1,1,0
1,10087938@N02,1,58,1205332532,1205332541,2,2,9
2,10087938@N02,1,66,1205342722,1205342729,2,2,7
3,10087938@N02,2,59,1205374109,1205374109,1,1,0
4,10087938@N02,3,58,1205417265,1205417265,1,1,0
5,10087938@N02,4,58,1205508764,1205538412,3,2,29648
6,10087938@N02,4,59,1205512653,1205514882,22,2,2229
7,10087938@N02,5,59,1205575764,1205595114,4,1,19350
8,10087938@N02,6,59,1205625724,1205641382,2,1,15658
9,10087938@N02,7,58,1205812892,1205812892,1,1,0


In [21]:
#extract trajectory, i.e., a list of POIs, considering loops/subtours:

def extract_traj_withloop(tid, visits):
    """Compute trajectories info, taking care of trajectories that contain sub-tours"""
    traj_df = visits[visits['trajID'] == tid].copy()
    traj_df.sort_values(by='dateTaken', ascending=True, inplace=True)
    traj = []
    for ix in traj_df.index:
        poi = traj_df.loc[ix, 'poiID']
        if len(traj) == 0:
            traj.append(poi)
        else:
            if poi != traj[-1]:
                traj.append(poi)
    return traj

In [22]:
def extract_traj(tid, traj_all):
    traj = traj_all[traj_all['trajID'] == tid].copy()
    traj.sort_values(by=['startTime'], ascending=True, inplace=True)
    return traj['poiID'].tolist()

In [23]:
loopcnt = 0
for tid_ in visits['trajID'].unique():
    traj_ = extract_traj_withloop(tid_, visits)
    if len(traj_) != len(set(traj_)):
        #print(traj_)
        loopcnt += 1
print('Number of trajectories with loops/subtours:', loopcnt)

Number of trajectories with loops/subtours: 211


In [24]:
#Compute POI properties, e.g., popularity, total number of visit, average visit duration.

def calc_poi_info(trajid_list, traj_all, poi_all):
    assert(len(trajid_list) > 0)
    # to allow duplicated trajid
    poi_info = traj_all[traj_all['trajID'] == trajid_list[0]][['poiID', 'poiDuration']].copy() 
    for i in range(1, len(trajid_list)):
        traj = traj_all[traj_all['trajID'] == trajid_list[i]][['poiID', 'poiDuration']]
        poi_info = poi_info.append(traj, ignore_index=True)
    
    poi_info = poi_info.groupby('poiID').agg([np.mean, np.size])
    poi_info.columns = poi_info.columns.droplevel()
    poi_info.reset_index(inplace=True)
    poi_info.rename(columns={'mean':'avgDuration', 'size':'nVisit'}, inplace=True)
    poi_info.set_index('poiID', inplace=True) 
    poi_info['poiCat'] = poi_all.loc[poi_info.index, 'poiTheme']
    poi_info['poiLon'] = poi_all.loc[poi_info.index, 'poiLon']
    poi_info['poiLat'] = poi_all.loc[poi_info.index, 'poiLat']
    
    # POI popularity: the number of distinct users that visited the POI
    pop_df = traj_all[traj_all['trajID'].isin(trajid_list)][['poiID', 'userID']].copy()
    pop_df = pop_df.groupby('poiID').agg(pd.Series.nunique)
    pop_df.rename(columns={'userID':'nunique'}, inplace=True)
    poi_info['popularity'] = pop_df.loc[poi_info.index, 'nunique']
    
    return poi_info.copy()

In [25]:
#compute the F1 score for recommended trajectory

def calc_F1(traj_act, traj_rec):
    '''Compute recall, precision and F1 for recommended trajectories'''
    assert(isinstance(noloop, bool))
    assert(len(traj_act) > 0)
    assert(len(traj_rec) > 0)
    
    intersize = len(set(traj_act) & set(traj_rec))
    recall = intersize / len(traj_act)
    precision = intersize / len(traj_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

In [26]:
#compute distance between two POIs using the Haversine formula

def calc_dist_vec(longitudes1, latitudes1, longitudes2, latitudes2):
    """Calculate the distance (unit: km) between two places on earth, vectorised"""
    # convert degrees to radians
    lng1 = np.radians(longitudes1)
    lat1 = np.radians(latitudes1)
    lng2 = np.radians(longitudes2)
    lat2 = np.radians(latitudes2)
    radius = 6371.0088 # mean earth radius

    dlng = np.fabs(lng1 - lng2)
    dlat = np.fabs(lat1 - lat2)
    dist =  2 * radius * np.arcsin( np.sqrt( 
                (np.sin(0.5*dlat))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(0.5*dlng))**2 ))
    return dist

In [27]:
#distance between POIs:

POI_DISTMAT = pd.DataFrame(data=np.zeros((poi_all.shape[0], poi_all.shape[0]), dtype=np.float), \
                           index=poi_all.index, columns=poi_all.index)

In [28]:
for ix in poi_all.index:
    POI_DISTMAT.loc[ix] = calc_dist_vec(poi_all.loc[ix, 'poiLon'], \
                                        poi_all.loc[ix, 'poiLat'], \
                                        poi_all['poiLon'], \
                                        poi_all['poiLat'])

In [29]:
trajid_set_all = sorted(traj_all['trajID'].unique().tolist())

In [30]:
poi_info_all = calc_poi_info(trajid_set_all, traj_all, poi_all)

In [31]:
poi_info_all.head()

,avgDuration,nVisit,poiCat,poiLon,poiLat,popularity
poiID,,,,,,
0,336.986301,73,City precincts,144.96778,-37.82167,64
1,817.134615,52,City precincts,144.94600,-37.81700,39
2,1216.853448,116,City precincts,144.97300,-37.81190,83
3,1147.129630,54,City precincts,144.96694,-37.79972,44
4,1635.500000,84,City precincts,144.96333,-37.80778,50


In [32]:
#dump POI popularity.

poi_all['poiPopularity'] = 0
poi_all.loc[poi_info_all.index, 'poiPopularity'] = poi_info_all.loc[poi_info_all.index, 'popularity']
poi_all.head()

,poiName,poiTheme,poiLat,poiLon,poiURL,poiPopularity
poiID,,,,,,
0,Arts Precinct,City precincts,-37.82167,144.96778,https://en.wikipedia.org/wiki/Melbourne_Arts_P...,64
1,Docklands,City precincts,-37.81700,144.94600,"https://en.wikipedia.org/wiki/Docklands,_Victoria",39
2,Government Precinct,City precincts,-37.81190,144.97300,"https://en.wikipedia.org/wiki/Spring_Street,_M...",83
3,Little Italy,City precincts,-37.79972,144.96694,"https://en.wikipedia.org/wiki/Little_Italy,_Me...",44
4,RMIT City,City precincts,-37.80778,144.96333,https://en.wikipedia.org/wiki/City_campus_of_t...,50


In [31]:
#poi_all.to_csv('poi_all.csv', index=True)
#zero_duration = poi_info_all[poi_info_all['avgDuration'] < 1]
#zero_duration
#print(traj_all.shape)
#traj_all = traj_all[traj_all['poiID'].isin(set(poi_info_all.index) - set(zero_duration.index))]
#print(traj_all.shape)

In [33]:
#dictionary to map every trajectory ID to the actual trajectory.

traj_dict = dict()
for trajid in trajid_set_all:
    traj = extract_traj(trajid, traj_all)
    assert(trajid not in traj_dict)
    traj_dict[trajid] = traj

In [36]:
#query defined using tuple (start POI, end POI, #POI)

QUERY_ID_DICT = dict() 

keys = [(traj_dict[x][0], traj_dict[x][-1], len(traj_dict[x])) \
        for x in sorted(traj_dict.keys()) if len(traj_dict[x]) > 2]
cnt = 0
for key in keys:
    if key not in QUERY_ID_DICT:   # (start, end, length) --> qid
        QUERY_ID_DICT[key] = cnt
        cnt += 1

In [37]:
print('#traj in total:', len(trajid_set_all))

print('#traj (length > 2):', traj_all[traj_all['trajLen'] > 2]['trajID'].unique().shape[0])
print('#query tuple:', len(QUERY_ID_DICT))

#traj in total: 5106
#traj (length > 2): 442
#query tuple: 415


In [39]:
# 5. Recommendation via POI Ranking


#features used for ranking:
DF_COLUMNS = ['poiID', 'label', 'queryID', 'popularity', 'nVisit', 'avgDuration', \
              'sameCatStart', 'sameCatEnd', 'distStart', 'distEnd', 'trajLen', 'diffPopStart', \
              'diffPopEnd', 'diffNVisitStart', 'diffNVisitEnd', 'diffDurationStart', 'diffDurationEnd']

#generating training dataFrame
def gen_train_subdf(poi_id, query_id_set, poi_info, query_id_rdict):
    columns = DF_COLUMNS
    poi_distmat = POI_DISTMAT
    df_ = pd.DataFrame(data=np.zeros((len(query_id_set), len(columns)), dtype=np.float), columns=columns)
    
    pop = poi_info.loc[poi_id, 'popularity']; nvisit = poi_info.loc[poi_id, 'nVisit']
    cat = poi_info.loc[poi_id, 'poiCat']; duration = poi_info.loc[poi_id, 'avgDuration']
    
    for j in range(len(query_id_set)):
        qid = query_id_set[j]
        assert(qid in query_id_rdict) # qid --> (start, end, length)
        (p0, pN, trajLen) = query_id_rdict[qid]
        idx = df_.index[j]
        df_.loc[idx, 'poiID'] = poi_id
        df_.loc[idx, 'queryID'] = qid
        df_.loc[idx, 'popularity'] = pop
        df_.loc[idx, 'nVisit'] = nvisit
        df_.loc[idx, 'avgDuration'] = duration
        df_.loc[idx, 'sameCatStart'] = 1 if cat == poi_info.loc[p0, 'poiCat'] else -1
        df_.loc[idx, 'sameCatEnd']   = 1 if cat == poi_info.loc[pN, 'poiCat'] else -1
        df_.loc[idx, 'distStart'] = poi_distmat.loc[poi_id, p0]
        df_.loc[idx, 'distEnd']   = poi_distmat.loc[poi_id, pN]
        df_.loc[idx, 'trajLen'] = trajLen
        df_.loc[idx, 'diffPopStart'] = pop - poi_info.loc[p0, 'popularity']
        df_.loc[idx, 'diffPopEnd']   = pop - poi_info.loc[pN, 'popularity']
        df_.loc[idx, 'diffNVisitStart'] = nvisit - poi_info.loc[p0, 'nVisit']
        df_.loc[idx, 'diffNVisitEnd']   = nvisit - poi_info.loc[pN, 'nVisit']
        df_.loc[idx, 'diffDurationStart'] = duration - poi_info.loc[p0, 'avgDuration']
        df_.loc[idx, 'diffDurationEnd']   = duration - poi_info.loc[pN, 'avgDuration']
        
    return df_

In [40]:
def gen_train_df(trajid_list, traj_dict, poi_info, n_jobs=-1):
    columns = DF_COLUMNS
    poi_distmat = POI_DISTMAT
    query_id_dict = QUERY_ID_DICT
    train_trajs = [traj_dict[x] for x in trajid_list if len(traj_dict[x]) > 2]
    
    qid_set = sorted(set([query_id_dict[(t[0], t[-1], len(t))] for t in train_trajs]))
    poi_set = set()
    for tr in train_trajs:
        poi_set = poi_set | set(tr)
    
    #qid_poi_pair = list(itertools.product(qid_set, poi_set)) # Cartesian product of qid_set and poi_set
    #df_ = pd.DataFrame(data=np.zeros((len(qid_poi_pair), len(columns)), dtype= np.float), columns=columns)
    
    query_id_rdict = dict()
    for k, v in query_id_dict.items(): 
        query_id_rdict[v] = k  # qid --> (start, end, length)
    
    train_df_list = Parallel(n_jobs=n_jobs)\
                            (delayed(gen_train_subdf)(poi, qid_set, poi_info, query_id_rdict) \
                             for poi in poi_set)
                        
    assert(len(train_df_list) > 0)
    df_ = train_df_list[0]
    for j in range(1, len(train_df_list)):
        df_ = df_.append(train_df_list[j], ignore_index=True)            
        
    df_.set_index(['queryID', 'poiID'], inplace=True)
    for t in train_trajs:
        qid = query_id_dict[(t[0], t[-1], len(t))]
        for poi in t[1:-1]:  # do NOT count if the POI is startPOI/endPOI
            df_.loc[(qid, poi), 'label'] += 1

    df_.reset_index(inplace=True)
    return df_

In [41]:
#generating test dataframe
def gen_test_df(startPOI, endPOI, nPOI, poi_info):
    columns = DF_COLUMNS
    poi_distmat = POI_DISTMAT
    query_id_dict = QUERY_ID_DICT
    key = (p0, pN, trajLen) = (startPOI, endPOI, nPOI)
    assert(key in query_id_dict)
    assert(p0 in poi_info.index)
    assert(pN in poi_info.index)
    
    df_ = pd.DataFrame(data=np.zeros((poi_info.shape[0], len(columns)), dtype= np.float), columns=columns)
    poi_list = sorted(poi_info.index)
    
    qid = query_id_dict[key]
    df_['queryID'] = qid
    df_['label'] = np.random.rand(df_.shape[0]) # label for test data is arbitrary according to libsvm FAQ

    for i in range(df_.index.shape[0]):
        poi = poi_list[i]
        lon = poi_info.loc[poi, 'poiLon']; lat = poi_info.loc[poi, 'poiLat']
        pop = poi_info.loc[poi, 'popularity']; nvisit = poi_info.loc[poi, 'nVisit']
        cat = poi_info.loc[poi, 'poiCat']; duration = poi_info.loc[poi, 'avgDuration']
        idx = df_.index[i]
        df_.loc[idx, 'poiID'] = poi 
        df_.loc[idx, 'popularity'] = pop
        df_.loc[idx, 'nVisit'] = nvisit
        df_.loc[idx, 'avgDuration'] = duration
        df_.loc[idx, 'sameCatStart'] = 1 if cat == poi_info.loc[p0, 'poiCat'] else -1
        df_.loc[idx, 'sameCatEnd']   = 1 if cat == poi_info.loc[pN, 'poiCat'] else -1
        df_.loc[idx, 'distStart'] = poi_distmat.loc[poi, p0]
        df_.loc[idx, 'distEnd']   = poi_distmat.loc[poi, pN]
        df_.loc[idx, 'trajLen'] = trajLen
        df_.loc[idx, 'diffPopStart'] = pop - poi_info.loc[p0, 'popularity']
        df_.loc[idx, 'diffPopEnd']   = pop - poi_info.loc[pN, 'popularity']
        df_.loc[idx, 'diffNVisitStart'] = nvisit - poi_info.loc[p0, 'nVisit']
        df_.loc[idx, 'diffNVisitEnd']   = nvisit - poi_info.loc[pN, 'nVisit']
        df_.loc[idx, 'diffDurationStart'] = duration - poi_info.loc[p0, 'avgDuration']
        df_.loc[idx, 'diffDurationEnd']   = duration - poi_info.loc[pN, 'avgDuration']
    return df_

In [43]:
#ranking POIs using rankSVM
#use softmax function to convert ranking scores to a probability distribution
def softmax(x):
    x1 = x.copy()
    x1 -= np.max(x1)  # numerically more stable, REF: http://cs231n.github.io/linear-classify/#softmax
    expx = np.exp(x1)
    return expx / np.sum(expx, axis=0) # column-wise sum

In [44]:
# python wrapper of rankSVM
class RankSVM:
    def __init__(self, bin_dir, useLinear=True, debug=False):
        dir_ = !echo $bin_dir  
        assert(os.path.exists(dir_[0]))
        self.bin_dir = dir_[0]
        
        self.bin_train = 'svm-train'
        self.bin_predict = 'svm-predict'
        if useLinear:
            self.bin_train = 'train'
            self.bin_predict = 'predict'
        
        assert(isinstance(debug, bool))
        self.debug = debug
        
        self.fmodel = None
        self.fscale = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            self.fmodel = fd.name
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            self.fscale = fd.name
        
        if self.debug:
            print('model file:', self.fmodel)
            print('feature scaling parameter file:', self.fscale)
    
    
    def __del__(self):
        if self.fmodel is not None and os.path.exists(self.fmodel):
            os.unlink(self.fmodel)
        if self.fscale is not None and os.path.exists(self.fscale):
            os.unlink(self.fscale)
    
    
    def train(self, train_df, cost=1):
        ftrain = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftrain = fd.name
            datastr = gen_data_str(train_df)
            fd.write(datastr)
        
        # feature scaling
        ftrain_scaled = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftrain_scaled = fd.name
        result = !$self.bin_dir/svm-scale -s $self.fscale $ftrain > $ftrain_scaled
        
        if self.debug:
            print('cost:', cost)
            print('train data file:', ftrain)
            print('feature scaled train data file:', ftrain_scaled)
        

        result = !$self.bin_dir/$self.bin_train -c $cost $ftrain_scaled $self.fmodel
        if self.debug:
            print('Training finished.')
            for i in range(len(result)): print(result[i])

        os.unlink(ftrain)
        os.unlink(ftrain_scaled)        
        
    
    def predict(self, test_df):
        # predict ranking scores for the given feature matrix
        if self.fmodel is None or not os.path.exists(self.fmodel):
            print('Model should be trained before predicting')
            return
        
        # write test data to file
        ftest = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftest = fd.name
            datastr = gen_data_str(test_df)
            fd.write(datastr)
                
        # feature scaling
        ftest_scaled = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            ftest_scaled = fd.name
        result = !$self.bin_dir/svm-scale -r $self.fscale $ftest > $ftest_scaled
            
        # generate prediction file
        fpredict = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            fpredict = fd.name
            
        if self.debug:
            print('test data file:', ftest)
            print('feature scaled test data file:', ftest_scaled)
            print('predict result file:', fpredict)
            
        # predict using trained model and write prediction to file
        result = !$self.bin_dir/$self.bin_predict $ftest_scaled $self.fmodel $fpredict
        if self.debug:
            print('Predict result: %-30s  %s' % (result[0], result[1]))
        
        # generate prediction DataFrame from prediction file
        poi_rank_df = pd.read_csv(fpredict, header=None)
        poi_rank_df.rename(columns={0:'rank'}, inplace=True)
        poi_rank_df['poiID'] = test_df['poiID'].astype(np.int)
        poi_rank_df.set_index('poiID', inplace=True) # duplicated 'poiID' when evaluating training data
        #poi_rank_df['probability'] = softmax(poi_rank_df['rank'])  # softmax
        
        # remove test file and prediction file
        os.unlink(ftest)
        os.unlink(ftest_scaled)
        os.unlink(fpredict)
        
        return poi_rank_df

Choose a trajectory of length 4.

In [106]:
#6. recommendation results comparison & visualisation

#trajectory of length 4
traj4s = traj_all[traj_all['trajLen'] == 4]['trajID'].unique()
traj4s

array([  64,  147,  150,  198,  311,  385,  445,  680,  720,  725,  765,
        771,  785,  835,  841,  902,  966, 1042, 1087, 1119, 1134, 1150,
       1170, 1208, 1217, 1233, 1241, 1245, 1265, 1277, 1315, 1562, 1593,
       1769, 1906, 1914, 1921, 1923, 1936, 1984, 1990, 2042, 2101, 2144,
       2220, 2240, 2301, 2340, 2354, 2430, 2572, 2646, 2672, 2690, 2826,
       2912, 3004, 3017, 3091, 3271, 3408, 3505, 3659, 3745, 3775, 3811,
       3888, 3891, 4124, 4130, 4133, 4160, 4183, 4198, 4202, 4204, 4205,
       4245, 4316, 4368, 4372, 4376, 4396, 4440, 4450, 4702, 4710, 4742,
       4832, 4842, 4887, 4906, 4922, 5064, 5067])

In [107]:
#for tid in traj4s:
#    gen_kml(tid, traj_all, poi_df)

In [108]:
#trajectory 680 to illustrate:
tid = 680
traj = extract_traj(tid, traj_all)
print('REAL:', traj)

REAL: [76, 81, 71, 68]


In [109]:
traj_dict_rec = {'REAL_' + str(tid): traj}

In [110]:
start = traj[0]
end = traj[-1]
length = len(traj)

In [111]:
# recommendation by POI Popularity
poi_df.sort_values(by='#distinctUsers', ascending=False, inplace=True)
rec_pop = [start] + [x for x in poi_df.index.tolist() if x not in {start, end}][:length-2] + [end]
print('REC_POP:', rec_pop)

REC_POP: [76, 71, 82, 68]
